In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import re


def create_currency_table():
    pattern = re.compile('\d{2}[./-]\d{2}[./-]\d{4}')  # dd/mm/yy
    while True:
        date = input('Введите дату')
        if re.match(pattern=pattern, string=date):
            break
        print('Неверный формат ввода даты, попробуйте еще раз')

    url = 'https://cbr.ru/scripts/XML_daily.asp?date_req='
    date = '/'.join(re.split('[./-]', date))

    def get_soup(url, date):
        response = requests.get(url + date)
        soup = BeautifulSoup(response.content, "xml")
        return soup

    soup1 = get_soup(url, date)
    info = {'currency_code': list(map(lambda x: x.text, soup1.find_all('CharCode'))),
            'currency_number': list(map(lambda x: x.text, soup1.find_all('NumCode'))),
            'currency_name': list(map(lambda x: x.text, soup1.find_all('Name'))),
            'currency_nominal_thisday': list(map(lambda x: float(x.text), soup1.find_all('Nominal'))),
            'currency_value_for_nominal_thisday': list(
                map(lambda x: float(x.text.replace(',', '.')), soup1.find_all('Value')))}

    # datetime -  новое
    def todateformat(date, split_sign='/', join_sign='-', change_type=False):
        reformated = join_sign.join(date.split(split_sign)[::-1])
        if change_type:
            return datetime.date.fromisoformat(reformated)
        return reformated

    thisday = todateformat(date, change_type=True)

    def find_the_day_before(date):
        return str(date - datetime.timedelta(days=1))

    before = todateformat(find_the_day_before(thisday), split_sign='-', join_sign='/')

    soup2 = get_soup(url, before)
    info['currency_value_for_nominal_before'] = list(
        map(lambda x: float(x.text.replace(',', '.')), soup2.find_all('Value')))

    # работа с pandas
    df = pd.DataFrame(info)
    df['currency_value_for_one_unit_thisday'] = (df['currency_value_for_nominal_thisday'] /
                                                df['currency_nominal_thisday'])

    df['currency_value_for_one_unit_before'] = (df['currency_value_for_nominal_before'] /
                                               df['currency_nominal_thisday'])
    df['currency_value_dinamics'] = (df['currency_value_for_one_unit_thisday'] /
                                     df['currency_value_for_one_unit_before']) - 1


    def result(perc):
        if perc < 0:
            return 'depreciated'
        elif perc > 0:
            return 'appreciated'
        else:
            return "didn't change"


    df['dinamics_result'] = df['currency_value_dinamics'].apply(result)
    return df

df = create_currency_table()

In [ ]:
# Вывод таблицы
df

In [ ]:
# Сохранение файла в виде Excel таблицы
path = input()
filename = input()
df.to_excel(f'{path + filename}.xlsx')